In [1]:
"""Research Tools.

This module provides search and content processing utilities for the research agent,
including web search capabilities and content summarization tools.
"""
import os
from datetime import datetime
import uuid, base64

import httpx
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import InjectedToolArg, InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState
from langgraph.types import Command
from markdownify import markdownify
from pydantic import BaseModel, Field
from tavily import TavilyClient
from typing_extensions import Annotated, Literal
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

from prompts import (
    RESEARCHER_INSTRUCTIONS,
    TODO_USAGE_INSTRUCTIONS,
    FILE_USAGE_INSTRUCTIONS,
    SUBAGENT_USAGE_INSTRUCTIONS,
)

from deepagents import DeepAgentState
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

# Summarization model 
# summarization_model = init_chat_model(model="azure_ai:gpt-5-mini")
summarization_model = AzureAIChatCompletionsModel(
    credential=os.getenv("AZURE_CREDENTIAL"),
    endpoint=os.getenv("AZURE_ENDPOINT"),
    model="gpt-5-mini",
)

tavily_client = TavilyClient()

class Summary(BaseModel):
    """Schema for webpage content summarization."""
    filename: str = Field(description="Name of the file to store.")
    summary: str = Field(description="Key learnings from the webpage.")

def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

def run_tavily_search(
    search_query: str, 
    max_results: int = 1, 
    topic: Literal["general", "news", "finance"] = "general", 
    include_raw_content: bool = True, 
) -> dict:
    """Perform search using Tavily API for a single query.

    Args:
        search_query: Search query to execute
        max_results: Maximum number of results per query
        topic: Topic filter for search results
        include_raw_content: Whether to include raw webpage content

    Returns:
        Search results dictionary
    """
    result = tavily_client.search(
        search_query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic
    )

    return result

SUMMARIZE_WEB_SEARCH = """You are creating a minimal summary for research steering - your goal is to help an agent know what information it has collected, NOT to preserve all details.

<webpage_content>
{webpage_content}
</webpage_content>

Create a VERY CONCISE summary focusing on:
1. Main topic/subject in 1-2 sentences
2. Key information type (facts, tutorial, news, analysis, etc.)  
3. Most significant 1-2 findings or points

Keep the summary under 150 words total. The agent needs to know what's in this file to decide if it should search for more information or use this source.

Generate a descriptive filename that indicates the content type and topic (e.g., "mcp_protocol_overview.md", "ai_safety_research_2024.md").

Output format:
```json
{{
   "filename": "descriptive_filename.md",
   "summary": "Very brief summary under 150 words focusing on main topic and key findings"
}}
```

Today's date: {date}
"""

def summarize_webpage_content(webpage_content: str) -> Summary:
    """Summarize webpage content using the configured summarization model.
    
    Args:
        webpage_content: Raw webpage content to summarize
        
    Returns:
        Summary object with filename and summary
    """
    try:
        # Set up structured output model for summarization
        structured_model = summarization_model.with_structured_output(Summary)
        
        # Generate summary
        summary_and_filename = structured_model.invoke([
            HumanMessage(content=SUMMARIZE_WEB_SEARCH.format(
                webpage_content=webpage_content, 
                date=get_today_str()
            ))
        ])
        
        return summary_and_filename
        
    except Exception:
        # Return a basic summary object on failure
        return Summary(
            filename="search_result.md",
            summary=webpage_content[:1000] + "..." if len(webpage_content) > 1000 else webpage_content
        )

def process_search_results(results: dict) -> list[dict]:
    """Process search results by summarizing content where available.
    
    Args:
        results: Tavily search results dictionary
        
    Returns:
        List of processed results with summaries
    """
    processed_results = []

    # Create a client for HTTP requests
    HTTPX_CLIENT = httpx.Client()
    
    for result in results.get('results', []):
        
        # Get url 
        url = result['url']
        
        # Read url
        response = HTTPX_CLIENT.get(url)

        if response.status_code == 200:
            # Convert HTML to markdown
            raw_content = markdownify(response.text)
            summary_obj = summarize_webpage_content(raw_content)
        else:
            # Use Tavily's generated summary
            raw_content = result.get('raw_content', '')
            summary_obj = Summary(
                filename="URL_error.md",
                summary=result.get('content', 'Error reading URL; try another search.')
            )
        
        # uniquify file names
        uid = base64.urlsafe_b64encode(uuid.uuid4().bytes).rstrip(b"=").decode("ascii")[:8]
        name, ext = os.path.splitext(summary_obj.filename)
        summary_obj.filename = f"{name}_{uid}{ext}"

        processed_results.append({
            'url': result['url'],
            'title': result['title'],
            'summary': summary_obj.summary,
            'filename': summary_obj.filename,
            'raw_content': raw_content,
        })
    
    return processed_results

@tool(parse_docstring=True)
def tavily_search(
    query: str,
    state: Annotated[DeepAgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId],
    max_results: Annotated[int, InjectedToolArg] = 1,
    topic: Annotated[Literal["general", "news", "finance"], InjectedToolArg] = "general",
) -> Command:
    """Search web and save detailed results to files while returning minimal context.

    Performs web search and saves full content to files for context offloading.
    Returns only essential information to help the agent decide on next steps.

    Args:
        query: Search query to execute
        state: Injected agent state for file storage
        tool_call_id: Injected tool call identifier
        max_results: Maximum number of results to return (default: 1)
        topic: Topic filter - 'general', 'news', or 'finance' (default: 'general')

    Returns:
        Command that saves full results to files and provides minimal summary
    """
    # Execute search
    search_results = run_tavily_search(
        query,
        max_results=max_results,
        topic=topic,
        include_raw_content=True,
    ) 

    # Process and summarize results
    processed_results = process_search_results(search_results)
    
    # Save each result to a file and prepare summary
    files = state.get("files", {})
    saved_files = []
    summaries = []
    
    for i, result in enumerate(processed_results):
        # Use the AI-generated filename from summarization
        filename = result['filename']
        
        # Create file content with full details
        file_content = f"""# Search Result: {result['title']}

        **URL:** {result['url']}
        **Query:** {query}
        **Date:** {get_today_str()}

        ## Summary
        {result['summary']}

        ## Raw Content
        {result['raw_content'] if result['raw_content'] else 'No raw content available'}
        """
        
        files[filename] = file_content
        saved_files.append(filename)
        summaries.append(f"- {filename}: {result['summary']}...")
    
    # Create minimal summary for tool message - focus on what was collected
    summary_text = f"""🔍 Found {len(processed_results)} result(s) for '{query}':

    {chr(10).join(summaries)}

    Files: {', '.join(saved_files)}
    💡 Use read_file() to access full details when needed."""

    return Command(
        update={
            "files": files,
            "messages": [
                ToolMessage(summary_text, tool_call_id=tool_call_id)
            ],
        }
    )

@tool(parse_docstring=True)
def think_tool(reflection: str) -> str:
    """Tool for strategic reflection on research progress and decision-making.

    Use this tool after each search to analyze results and plan next steps systematically.
    This creates a deliberate pause in the research workflow for quality decision-making.

    When to use:
    - After receiving search results: What key information did I find?
    - Before deciding next steps: Do I have enough to answer comprehensively?
    - When assessing research gaps: What specific information am I still missing?
    - Before concluding research: Can I provide a complete answer now?
    - How complex is the question: Have I reached the number of search limits?

    Reflection should address:
    1. Analysis of current findings - What concrete information have I gathered?
    2. Gap assessment - What crucial information is still missing?
    3. Quality evaluation - Do I have sufficient evidence/examples for a good answer?
    4. Strategic decision - Should I continue searching or provide my answer?

    Args:
        reflection: Your detailed reflection on research progress, findings, gaps, and next steps

    Returns:
        Confirmation that reflection was recorded for decision-making
    """
    return f"Reflection recorded: {reflection}"


def run_agent(user_question: str):
    """Run the research agent with tools and sub-agent capabilities."""
    import os
    from datetime import datetime
    from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

    # Create agent using create_react_agent directly
    os.environ["AZURE_INFERENCE_ENDPOINT"]  = os.getenv("AZURE_ENDPOINT")
    os.environ["AZURE_INFERENCE_CREDENTIAL"] = os.getenv("AZURE_CREDENTIAL")

        # Create agent using create_react_agent directly
    model = AzureAIChatCompletionsModel(
            credential=os.getenv("AZURE_CREDENTIAL"),
            endpoint=os.getenv("AZURE_ENDPOINT"),
            model="gpt-5-mini",
        )

    # Limits
    max_concurrent_research_units = 3
    max_researcher_iterations = 3

    # Tools
    sub_agent_tools = [tavily_search, think_tool]


    # Create research sub-agent
    research_sub_agent = {
        "name": "research-sub-agent", 
        "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
        "prompt": RESEARCHER_INSTRUCTIONS.format(date=get_today_str()),
        "tools": ["tavily_search", "think_tool"],
    }


    # Build prompt
    SUBAGENT_INSTRUCTIONS = SUBAGENT_USAGE_INSTRUCTIONS.format(
        max_concurrent_research_units=max_concurrent_research_units,
        max_researcher_iterations=max_researcher_iterations,
        date=datetime.now().strftime("%a %b %-d, %Y"),
    )

    INSTRUCTIONS = (
        "# TODO MANAGEMENT\n"
        + TODO_USAGE_INSTRUCTIONS
        + "\n\n"
        + "=" * 80
        + "\n\n"
        + "# FILE SYSTEM USAGE\n"
        + FILE_USAGE_INSTRUCTIONS
        + "\n\n"
        + "=" * 80
        + "\n\n"
        + "# SUB-AGENT DELEGATION\n"
        + SUBAGENT_INSTRUCTIONS
    )

    # Create agent
    from deepagents import create_deep_agent

    agent = create_deep_agent(
        sub_agent_tools,
        INSTRUCTIONS,
        subagents=[research_sub_agent],
        model=model,
    )

    final_chunk = ""

    for chunk in agent.stream(
        input = {"messages": [{"role": "user", "content": f"{user_question}"}]},
        stream_mode="values"
    ):
        if "messages" in chunk:
            chunk["messages"][-1].pretty_print()
            final_chunk += chunk["messages"][-1].content
        else:
            print(chunk)
            if "messages" in chunk:
                final_chunk += chunk["messages"][-1].content
           

    return final_chunk


if __name__ == "__main__":
    run_agent("""Compare the fares for a round-trip flight from London (LHR) 
              to New York on 2025-11-20, from two different airlines, 
              including taxes and fees. Return the results as json with the
                following fields: 
                origin: str,
                destination: str,
                departure_date: str,
                return_date: str 
              don't ask for clarifications, just make your best assumptions. 
              IMPORTANT: You should use subagents to run tasks in parrallel when you can!

              """)

================================ Human Message =================================

Compare the fares for a round-trip flight from London (LHR) 
              to New York on 2025-11-20, from two different airlines, 
              including taxes and fees. Return the results as json with the
                following fields: 
                origin: str,
                destination: str,
                departure_date: str,
                return_date: str 
              don't ask for clarifications, just make your best assumptions. 
              IMPORTANT: You should use subagents to run tasks in parrallel when you can!

              
================================== Ai Message ==================================
Tool Calls:
  write_todos (call_JYQY7UmuBR1UIIS6wEQ9kRnA)
 Call ID: call_JYQY7UmuBR1UIIS6wEQ9kRnA
  Args:
    todos: [{'content': 'Research total round-trip fare (including taxes/fees) for British Airways from LHR to New York (assume JFK) departing 2025-11-20 returning 2025-1